In [2]:
import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
import pandas as pd
import time
import os
import multiprocessing as mp # for speeding up some process
import sys
sys.path.append('../py_model')
from utils import init_logging
import logging 
from scipy.cluster import hierarchy
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [16, 10] # customizing
from ycimpute.imputer.knnimput import KNN # inputation library


In [2]:
# base features
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index', 'TARGET']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
df.shape

(356251, 279)

In [3]:
discrete_features = []
continuous_features = []
for f in df.columns.tolist()[:]:
    if df[f].value_counts().size < 20:
        discrete_features.append(f)
    else:
        continuous_features.append(f)
len(discrete_features), len(continuous_features)

(71, 208)

In [4]:
'SK_ID_CURR' in discrete_features, 'TARGET' in discrete_features, 'index' in discrete_features

(False, False, False)

# MICE

In [19]:
df = pd.read_csv('../features/peeking_target_for_imputing_w_validating_0.61231807923718_auc.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [20]:
'TARGET' in df.columns

True

In [21]:
X_missing = df.copy()
# feature scaling before KNN
feature_scaling = True
if feature_scaling == True:
    for f in X_missing.columns.tolist():
        mean = X_missing[f].mean()
        std = X_missing[f].std()
        X_missing[f] = (X_missing[f] - mean) / std

print ('X_missing', X_missing.shape)
    
#-------------------
# core algorithm: input should be array
#-------------------
from fancyimpute import MICE # for imputing

logging.info('visit_sequence: {}'.format('monotone')) 
logging.info('impute_type: {}'.format('col')) 
logging.info('init_fill_method: {}'.format('mean')) 
logging.info('target == 1')
X_filled = MICE(visit_sequence = 'monotone', 
                impute_type = 'col',
                init_fill_method = 'mean').complete(X_missing.values)
X_filled = pd.DataFrame(X_filled, columns = X_missing.columns)

X_missing (356251, 72)
[MICE] Completing matrix with shape (356251, 72)
[MICE] Starting imputation round 1/110, elapsed time 0.312
[MICE] Starting imputation round 2/110, elapsed time 13.317
[MICE] Starting imputation round 3/110, elapsed time 25.788
[MICE] Starting imputation round 4/110, elapsed time 38.973
[MICE] Starting imputation round 5/110, elapsed time 52.285
[MICE] Starting imputation round 6/110, elapsed time 65.201
[MICE] Starting imputation round 7/110, elapsed time 78.365
[MICE] Starting imputation round 8/110, elapsed time 91.587
[MICE] Starting imputation round 9/110, elapsed time 105.155
[MICE] Starting imputation round 10/110, elapsed time 118.540
[MICE] Starting imputation round 11/110, elapsed time 131.810
[MICE] Starting imputation round 12/110, elapsed time 145.321
[MICE] Starting imputation round 13/110, elapsed time 158.463
[MICE] Starting imputation round 14/110, elapsed time 171.853
[MICE] Starting imputation round 15/110, elapsed time 185.406
[MICE] Starting 

In [22]:
# base features
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index', 'TARGET']
df.drop(no_need_to_inpute, axis = 1, inplace = True)

for f_in_same_cluster in X_filled.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled[f_in_same_cluster]
print (df.shape)
# if 'TARGET' in df.columns.tolist():
#     df.drop(['TARGET'], axis = 1, inplace = True)

# df = pd.merge(df, copy_for_the_following_merge, on = 'SK_ID_CURR', how = 'left')
# print ('final', df.shape)

featurs AMT_REQ_CREDIT_BUREAU_QRT
featurs CODE_GENDER
featurs DEF_30_CNT_SOCIAL_CIRCLE
featurs DEF_60_CNT_SOCIAL_CIRCLE
featurs FLAG_DOCUMENT_11
featurs FLAG_DOCUMENT_18
featurs FLAG_DOCUMENT_3
featurs FLAG_DOCUMENT_8
featurs FLAG_OWN_CAR
featurs FLAG_PHONE
featurs FLAG_WORK_PHONE
featurs LIVE_CITY_NOT_WORK_CITY
featurs REGION_RATING_CLIENT
featurs REGION_RATING_CLIENT_W_CITY
featurs REG_CITY_NOT_LIVE_CITY
featurs TARGET
featurs NEW_DOC_IND_STD
featurs NEW_DOC_IND_KURT
featurs AMT_INCOME_TOTAL_BY_CODE_GENDER_NAME_EDUCATION_TYPE
featurs AMT_ANNUITY_BY_OCCUPATION_TYPE
featurs CNT_CHILDREN_BY_OCCUPATION_TYPE
featurs CNT_FAM_MEMBERS_BY_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_OCCUPATION_TYPE
featurs DAYS_EMPLOYED_BY_OCCUPATION_TYPE
featurs DAYS_ID_PUBLISH_BY_OCCUPATION_TYPE
featurs DAYS_REGISTRATION_BY_OCCUPATION_TYPE
featurs EXT_SOURCE_1_BY_OCCUPATION_TYPE
featurs EXT_SOURCE_2_BY_OCCUPATION_TYPE
featurs EXT_SOURCE_3_BY_OCCUPATION_TYPE
featurs NAME_CONTRACT_TYPE_Cash loans
featurs NAME_CONTRA

In [23]:
'TARGET' in df.columns.tolist(), 'SK_ID_CURR' in df.columns.tolist()

(True, True)

In [24]:
if 'TARGET' in df.columns.tolist():
    df.drop(['TARGET'], axis = 1, inplace = True)

df = pd.merge(df, copy_for_the_following_merge, on = 'SK_ID_CURR', how = 'left')
print ('final', df.shape)

final (356251, 280)


In [25]:
#-------------------
# save
#-------------------
output_path = '../features/filled_by_mice'
if not os.path.isdir(output_path):
    os.mkdir(output_path)

df.to_hdf(
    os.path.join(output_path, 'normalized_mice_discrete_w_peeking_target_612.h5'), 
    'normalized_mice_discrete_w_peeking_target_612')



In [26]:
os.listdir('../features/filled_by_mice')

['base_featurs_filled_mice_by_cluster_2+1.h5',
 'normalized_mice_discrete_wo_target.h5',
 'normalized_mice_discrete_w_peeking_target_724.h5',
 'normalized_mice_discrete.h5',
 'normalized_mice_similar_features_th_09_5.h5',
 'normalized_mice_cluster_3.h5',
 'base_featurs_filled_mice_by_cluster_2.h5',
 'normalized_mice_discrete_w_peeking_target.h5',
 '.DS_Store',
 'base_featurs_filled_mice_clustering.h5',
 'base_featurs_filled_mice_by_cluster_1.h5',
 'normalized_mice_cluster_1_2_3.h5',
 'normalized_mice_similar_features_th_09_2.h5',
 'normalized_mice_similar_features_th_09_1.h5',
 'normalized_mice_cluster_2.h5',
 'base_featurs_filled_mice_half_training_data.h5',
 'normalized_mice_cluster_1.h5',
 'normalized_mice_discrete_w_peeking_target_612.h5',
 'base_featurs_filled_mice_half_training_data_wo_target.h5']

In [ ]:
# base features
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
print (df.shape)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)
X_missing = df[continuous_features]

# feature scaling before KNN
feature_scaling = True
if feature_scaling == True:
    for f in X_missing.columns.tolist():
        mean = X_missing[f].mean()
        std = X_missing[f].std()
        X_missing[f] = (X_missing[f] - mean) / std

print ('X_missing', X_missing.shape)
    
#-------------------
# core algorithm: input should be array
#-------------------
from fancyimpute import MICE # for imputing

logging.info('visit_sequence: {}'.format('monotone')) 
logging.info('impute_type: {}'.format('col')) 
logging.info('init_fill_method: {}'.format('mean')) 
logging.info('target == 1')
X_filled = MICE(visit_sequence = 'monotone', 
                impute_type = 'col',
                init_fill_method = 'mean').complete(X_missing.values)
X_filled = pd.DataFrame(X_filled, columns = X_missing.columns)

for f_in_same_cluster in X_filled.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled[f_in_same_cluster]

if 'TARGET' in df.columns.tolist():
    df.drop(['TARGET'], axis = 1, inplace = True)

df = pd.merge(df, copy_for_the_following_merge, on = 'SK_ID_CURR', how = 'left')
print ('final', df.shape)

(356251, 281)


/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


X_missing (356251, 208)


Using TensorFlow backend.
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


[MICE] Completing matrix with shape (356251, 208)
[MICE] Starting imputation round 1/110, elapsed time 1.114
[MICE] Starting imputation round 2/110, elapsed time 336.723
[MICE] Starting imputation round 3/110, elapsed time 676.015
[MICE] Starting imputation round 4/110, elapsed time 1020.693
[MICE] Starting imputation round 5/110, elapsed time 1359.937
[MICE] Starting imputation round 6/110, elapsed time 1691.874
[MICE] Starting imputation round 7/110, elapsed time 2023.672
[MICE] Starting imputation round 8/110, elapsed time 2363.061
[MICE] Starting imputation round 9/110, elapsed time 2701.611
[MICE] Starting imputation round 10/110, elapsed time 3033.714
[MICE] Starting imputation round 11/110, elapsed time 3366.829
[MICE] Starting imputation round 12/110, elapsed time 3701.154
[MICE] Starting imputation round 13/110, elapsed time 4039.028
[MICE] Starting imputation round 14/110, elapsed time 4372.514
[MICE] Starting imputation round 15/110, elapsed time 4708.013
[MICE] Starting imp

In [ ]:
#-------------------
# save
#-------------------
output_path = '../features/filled_by_mice'
if not os.path.isdir(output_path):
    os.mkdir(output_path)

df.to_hdf(
    os.path.join(output_path, 'normalized_mice_continue.h5'), 
    'normalized_mice_continue')


In [ ]:
# base features
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
X_missing1 = df[discrete_features]
# feature scaling before KNN
feature_scaling = True
if feature_scaling == True:
    for f in X_missing1.columns.tolist():
        mean = X_missing1[f].mean()
        std = X_missing1[f].std()
        X_missing1[f] = (X_missing1[f] - mean) / std

print ('X_missing', X_missing1.shape)
    
#-------------------
# core algorithm: input should be array
#-------------------
from fancyimpute import MICE # for imputing

logging.info('visit_sequence: {}'.format('monotone')) 
logging.info('impute_type: {}'.format('col')) 
logging.info('init_fill_method: {}'.format('mean')) 
logging.info('target == 1')
X_filled1 = MICE(visit_sequence = 'monotone', 
                impute_type = 'col',
                init_fill_method = 'mean').complete(X_missing1.values)
X_filled1 = pd.DataFrame(X_filled1, columns = X_missing1.columns)

# base features
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
X_missing2 = df[discrete_features]
# feature scaling before KNN
feature_scaling = True
if feature_scaling == True:
    for f in X_missing2.columns.tolist():
        mean = X_missing2[f].mean()
        std = X_missing2[f].std()
        X_missing2[f] = (X_missing2[f] - mean) / std

print ('X_missing', X_missing2.shape)
    
#-------------------
# core algorithm: input should be array
#-------------------
from fancyimpute import MICE # for imputing

logging.info('visit_sequence: {}'.format('monotone')) 
logging.info('impute_type: {}'.format('col')) 
logging.info('init_fill_method: {}'.format('mean')) 
logging.info('target == 1')
X_filled2 = MICE(visit_sequence = 'monotone', 
                impute_type = 'col',
                init_fill_method = 'mean').complete(X_missing2.values)
X_filled2 = pd.DataFrame(X_filled2, columns = X_missing2.columns)


for f_in_same_cluster in X_filled1.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled1[f_in_same_cluster]
    
for f_in_same_cluster in X_filled2.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled2[f_in_same_cluster]
    
if 'TARGET' in df.columns.tolist():
    df.drop(['TARGET'], axis = 1, inplace = True)

df = pd.merge(df, copy_for_the_following_merge, on = 'SK_ID_CURR', how = 'left')
print ('final', df.shape)

In [ ]:
#-------------------
# save
#-------------------
output_path = '../features/filled_by_mice'
if not os.path.isdir(output_path):
    os.mkdir(output_path)

df.to_hdf(
    os.path.join(output_path, 'normalized_mice_discrtete_continue.h5'), 
    'normalized_mice_discrtete_continue')


# KNN

In [ ]:
feature_scaling = True
X_missing_df = df[discrete_features]
# feature scaling before KNN
if feature_scaling == True:
    for f in X_missing_df.columns.tolist():
        mean = X_missing_df[f].mean()
        std = X_missing_df[f].std()
        X_missing_df[f] = (X_missing_df[f] - mean) / std

#----------------------------
# KNN imputing
#----------------------------
small_df_ls = []
previous_step = None
num_split = 20
for i, step in enumerate(np.arange(0, X_missing_df.shape[0], step = int(X_missing_df.shape[0] / num_split))):
    # for memory problem, we cannot feed all the data points into algorithm, which depends on row and n_features
    if i == 0:
        pass
    elif i == 1:
        small_df_ls.append(X_missing_df[0: step])
        previous_step = step
    elif i == (num_split):
        small_df_ls.append(X_missing_df[previous_step: ])
        break
    else:
        small_df_ls.append(X_missing_df[previous_step: step])
        previous_step = step
'splitting is okay? {}'.format(pd.concat(small_df_ls, axis = 0).shape)

In [ ]:
def multi(small_df):
    X_missing = small_df.copy()
    print ('input of KNN imputing : {}'.format(X_missing.shape))
    # core algorithm: input should be array
    X_filled = KNN(k = k).complete(X_missing.values)
    return pd.DataFrame(X_filled, columns = X_missing.columns)
mp_pool = mp.Pool(len(small_df_ls)) 
small_df_filled_ls = mp_pool.map(multi, small_df_ls)

In [ ]:
#----------------------------
# imputing
#----------------------------
k = 65
small_df_filled_ls = []
for small_df in small_df_ls:
    X_missing = small_df.copy()
    print ('input of KNN imputing : {}'.format(X_missing.shape))
    # core algorithm: input should be array
    X_filled = KNN(k = k).complete(X_missing.values)
    small_df_filled_ls.append(pd.DataFrame(X_filled, columns = X_missing.columns))

X_filled1 = pd.concat(small_df_filled_ls, axis = 0)
'X_filled1 : {}'.format(X_filled1.shape)

In [ ]:
'TARGET' in discrete_features, 'SK_ID_CURR' in discrete_features, 

In [ ]:
#----------------------------
# reload base_features for filling
#----------------------------
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
for f_in_same_cluster in X_filled1.columns.tolist():
    if f_in_same_cluster in set(df.columns.tolist()):
        #logging.info('featurs : {}'.format(f_in_same_cluster))
        df[f_in_same_cluster] = X_filled1[f_in_same_cluster].tolist()


In [ ]:
if 'SK_ID_CURR' in X_filled1.columns.tolist():
    df.drop(['SK_ID_CURR'], axis = 1, inplace = True)
if 'TARGET' in X_filled1.columns.tolist():
    df.drop(['TARGET'], axis = 1, inplace = True)
    
df = pd.merge(df, copy_for_the_following_merge, on = 'SK_ID_CURR', how = 'left')

In [ ]:
df.shape

In [ ]:
#-------------------
# save
#-------------------
output_path = '../features/filled_by_knn'
if not os.path.isdir(output_path):
    os.mkdir(output_path)

df.to_hdf(
    os.path.join(output_path, 'normalized_knn_65_descrete.h5'), 'normalized_knn_65_descrete')



# continuous features

In [ ]:
# base features
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)

feature_scaling = True
X_missing_df = df[continuous_features]
# feature scaling before KNN
if feature_scaling == True:
    for f in X_missing_df.columns.tolist():
        mean = X_missing_df[f].mean()
        std = X_missing_df[f].std()
        X_missing_df[f] = (X_missing_df[f] - mean) / std

#----------------------------
# KNN imputing
#----------------------------
small_df_ls = []
previous_step = None
num_split = 20
for i, step in enumerate(np.arange(0, X_missing_df.shape[0], step = int(X_missing_df.shape[0] / num_split))):
    # for memory problem, we cannot feed all the data points into algorithm, which depends on row and n_features
    if i == 0:
        pass
    elif i == 1:
        small_df_ls.append(X_missing_df[0: step])
        previous_step = step
    elif i == (num_split):
        small_df_ls.append(X_missing_df[previous_step: ])
        break
    else:
        small_df_ls.append(X_missing_df[previous_step: step])
        previous_step = step
'splitting is okay? {}'.format(pd.concat(small_df_ls, axis = 0).shape)

In [ ]:
def multi(small_df):
    k = 65
    print ('input of KNN imputing : {}'.format(small_df.shape))
    # core algorithm: input should be array
    X_filled = KNN(k = k).complete(small_df.values)
    return pd.DataFrame(X_filled, columns = small_df.columns)
mp_pool = mp.Pool(len(small_df_ls)) 
small_df_filled_ls = mp_pool.map(multi, small_df_ls)

In [ ]:
X_filled1 = pd.concat(small_df_filled_ls, axis = 0)
'X_filled1 : {}'.format(X_filled1.shape)
#----------------------------
# reload base_features for filling
#----------------------------
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
for f_in_same_cluster in X_filled1.columns.tolist():
    if f_in_same_cluster in set(df.columns.tolist()):
        #logging.info('featurs : {}'.format(f_in_same_cluster))
        df[f_in_same_cluster] = X_filled1[f_in_same_cluster].tolist()
        
if 'TARGET' in df.columns.tolist():
    df.drop(['TARGET'], axis = 1, inplace = True)

df = pd.merge(df, copy_for_the_following_merge, on = 'SK_ID_CURR', how = 'left')
print ('final', df.shape)

#-------------------
# save
#-------------------
output_path = '../features/filled_by_knn'
if not os.path.isdir(output_path):
    os.mkdir(output_path)

df.to_hdf(
    os.path.join(output_path, 'normalized_knn_65_continue.h5'), 'normalized_knn_65_continue')



In [5]:
df = pd.read_hdf('../features/filled_by_knn/normalized_knn_65_descrete.h5')

In [7]:
df[['TARGET']]

,TARGET
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0
